In [ ]:
import tensorflow as tf
import math
import numpy as np
def glu(x):
	n_units = tf.shape(x)[-1] // 2
	return x[..., :n_units] * tf.nn.sigmoid(x[..., n_units:])

def sinc(t):
  return tf.where(t == 0, tf.constant(1., dtype=t.dtype), tf.math.sin(t) / t)

"""Bi Lstm Layer"""
class BLSTM:
	def __init__(self, dim, num_layers=1):                                                       
		self.dim = dim
		self.num_layers = num_layers
		self.lstm = tf.keras.Sequential()
		# self.lstm = tf.keras.layers.LSTM(dim, input_shape=(31,768))
		# self.lstm2 = tf.keras.layers.LSTM(dim)
		# self.dense = tf.keras.layers.Dense(dim)
		for _ in range(num_layers):
			self.lstm.add(
				# tf.keras.layers.Bidirectional(
					tf.keras.layers.RNN(tf.keras.layers.LSTMCell(units = dim, activation = 'tanh', kernel_initializer=tf.keras.initializers.Constant(1), recurrent_initializer=tf.keras.initializers.Constant(1)), return_sequences = True)
				# )
			)
	def __call__(self, x):
		# x = self.lstm(x)
		return self.lstm(x)


"""upsampling and downsampling: change sample rate of the given audio with factor of 2 or 1/2"""
def kernel_for_upsample_and_downsample_2(zeros=56):
	win = tf.signal.hann_window(window_length = 4*zeros+1, periodic=False)
	# print(win)
	winodd = win[1::2]
	t = tf.linspace(-zeros+0.5, zeros-0.5, 2*zeros)
	t *= math.pi
	kernel = tf.reshape(sinc(t)*winodd, [-1, 1, 1])
	return kernel

def upsample2(x, zeros=56):
	batch, sample, ch = x.shape
	kernel = kernel_for_upsample_and_downsample_2(zeros)
	out = tf.reshape(x, [-1, sample, 1])
	# tf.print(x)
	# tf.print(kernel)
	out = tf.nn.conv1d(input=out, filters=kernel, stride=1, padding="SAME")
	# print(out)
	# out = out[:,1:,:]
	out = tf.reshape(out, [-1, sample, ch])
	y = tf.stack([x, out], axis=-2)
	# print(tf.reshape(y, [-1, sample * 2, 1]))
	return tf.reshape(y, [-1, sample * 2, 1])

def downsample2(x, zeros=56):
	if x.shape[-2] % 2 != 0:
		x = tf.pad(x, [[0, 0],[0, 1],[0, 0]], "CONSTANT")
	xeven = x[:, ::2,:]
	xodd = x[:, 1::2,:]
	# print(xeven)
	batch, sample, ch = xodd.shape
	kernel = kernel_for_upsample_and_downsample_2(zeros)
	# print(kernel.shape)
	# kernel = tf.reshape(kernel, shape=(112, 1, 1))
	xodd = tf.reshape(xodd, [-1, sample, 1])
	xodd = tf.pad(xodd, [[0, 0,], [1, 1], [0, 0]], "CONSTANT")
	# print(xodd)
	xodd = tf.nn.conv1d(input=xodd, filters=kernel, stride=1, padding="SAME")[:, :-2, :]
	# print(xodd)
	# xodd = xodd[:,:-1,:]
	out = xeven + tf.reshape(xodd, [-1, sample, ch])
	out = tf.reshape(out, [-1, sample, ch])
	out = out * 0.5
	return out

def valid_length(length, depth = 5, s = 4, k = 8):
	length = math.ceil(length * 4)
	for _ in range(depth):
		length = math.ceil((length - k) / s) + 1
		length = max(length, 1)
	for _ in range(depth):
		length = (length - 1) * s + k
	length = int(math.ceil(length / 4))
	return int(length)

In [ ]:
def Demucs(x, input_shape = 6, hidden=64, depth=5, sr=6):
	# assert depth == 5
	encoder = []
	decoder = []
	activation = tf.keras.layers.Lambda(glu)

	chin, chout = 1, 1
	for index in range(depth):
		encoder.append(
		tf.keras.Sequential(
			[
				tf.keras.layers.Conv1D(
					filters=hidden, kernel_size=8, strides=4, activation=tf.nn.relu, kernel_initializer=tf.keras.initializers.Constant(0.01)
				),
				tf.keras.layers.Conv1D(
					filters=hidden*2, kernel_size=1, strides=1, kernel_initializer=tf.keras.initializers.Constant(0.01)
				),
				activation
			]
			, name=f"encode_{index}"
		))
		decoder.append(tf.keras.Sequential(
			[
				tf.keras.layers.Conv1D(
					filters=hidden*2, kernel_size=1, strides=1
				),
				activation,
				tf.keras.layers.Conv1DTranspose(
					filters=chout, kernel_size=8, strides=4
				)
			]
			+ ([tf.keras.layers.ReLU()] if index else [])
			, name=f"decode_{index}"
		))
		chout, chin, hidden = hidden, hidden, hidden*2
	
	decoder = decoder[::-1]
	lstm = BLSTM(chin)

	# input_audio = tf.keras.layers.Input(shape=input_shape, name = "input_layer")
	# x = input_audio

	"""normalize"""
	# mono = tf.math.reduce_mean(x,axis=1,keepdims=True)
	std = tf.math.reduce_std(x, axis=1, keepdims=True)
	x = x / (1e-3 + std)
	# tf.print(x)
	length = x.shape[1]
	"""pad to have valid length"""	
	x = tf.pad(x, [[0, 0],[0, valid_length(length) - length],[0, 0]], "CONSTANT")
 
	"""upsample the samplerate by 4"""
	# tf.print(x)
	x = upsample2(x)
	x = upsample2(x)
	# tf.print(x)
	# print(tf.reduce_sum(x))
	# filter = tf.constant(np.ones((8, 1, 64)), dtype=float)
	# bias = tf.constant(np.ones(64), dtype=float)
	# y = tf.nn.conv1d(input=x, filters=filter, stride=4, padding="VALID")
	# filter = tf.constant(np.ones((1, 64, 128)), dtype=float)
	# y = tf.nn.conv1d(input=y, filters=filter, stride=1, padding="VALID")
	# activation = tf.keras.layers.Lambda(glu)
	# y = activation(y)
	# y = tf.nn.bias_add(y, bias)
	# y = tf.keras.activations.|relu(y)
	# print(x)
	# x = upsample2(x)

	"""encoder part"""
	skips = [] # used for skip connection
	for encode in encoder:
		x = encode(x)
		# np.set_printoptions(precision=10)
		# print(x)
		# print(x[:, :, 0])
		# break
		val = tf.identity(x)
		skips.append(val)
	# print(x)
	# tt = tf.keras.layers.RNN(tf.keras.layers.LSTMCell(units=1024, kernel_initializer=tf.keras.initializers.Constant(1), recurrent_initializer=tf.keras.initializers.Constant(1)), return_sequences = True)
	# tt2 = tf.keras.layers.RNN(tf.keras.layers.LSTMCell(units=1024, kernel_initializer=tf.keras.initializers.Constant(2), recurrent_initializer=tf.keras.initializers.Constant(2)), return_sequences = True)
	# y = tt(x)
	# y = tt2(x) 
	# print(y[:,:,0])
	x = lstm(x)
	print(x)
	# x = tf.keras.layers.LSTM(chin)(x)
 
	"""decoder part"""
	for decode in decoder:
		skip = skips.pop(-1)
		# print(skip.shape)
		# print("Before", skip)
		# print("After", skip[:,:x.shape[1],:])
		x = tf.add(x, skip)
		# print(x)
		# x = x + skip[:,:x.shape[1],:] # add skip connection
		x = decode(x)

	# print(x.shape)
	"""downsample the samplerate by 4"""
	x = downsample2(x)
	x = downsample2(x)
	# print(x.shape)
	# x = downsample2(x)

	x = x[:,:length,:]
	x = x*std # multiply with normalize factor
	
	# model = tf.keras.models.Model(input_audio, x, name='demucs')
	return x

In [ ]:
# x = np.array([0.23, 0.55, 0.65, 0.4, 0.33, 0.4])
x = np.ones((1, 160000, 1))
x = tf.constant(x, dtype=tf.float32)
# x = tf.reshape(x, [1, 6, 1])
out = Demucs(x)

tf.Tensor(
[[[0.7615942 0.7615942 0.7615942 ... 0.7615942 0.7615942 0.7615942]
  [0.9640276 0.9640276 0.9640276 ... 0.9640276 0.9640276 0.9640276]
  [0.9950547 0.9950547 0.9950547 ... 0.9950547 0.9950547 0.9950547]
  ...
  [1.        1.        1.        ... 1.        1.        1.       ]
  [1.        1.        1.        ... 1.        1.        1.       ]
  [1.        1.        1.        ... 1.        1.        1.       ]]], shape=(1, 624, 1024), dtype=float32)


In [ ]:
import tensorflow as tf
x = np.array( [[ 1, 2, 3],[ 4, 5, 6]] )
x = np.transpose(x)
x = tf.constant(x, dtype=float)
y = np.array([[[1, 2],[3, 4]],[[1, 1],[1, 1]]])
y = np.transpose(y, (2, 1, 0))
# print(y)
y = tf.constant(y, dtype=float)
# print(y.shape)
# y = tf.constant(np.array([1, 2, 2, 3, 1, 2, 2, 3]), dtype=float)
x = tf.reshape(x, (1, 3, 2))
# print(x[:,:,0])
# print(y)
# y = tf.reshape(y, (2, 2, 2))
out = tf.nn.conv1d(
    input = x,
    filters=y,
    stride = 1,
    padding = "VALID"
)
print(out)

x = tf.reshape(x, (1, 1, 3, 2))
y = tf.reshape(y, (1, 2, 2, 2))
# y = tf.transpose(y, perm=[3, 0, 1, 2])
print(y)
out = tf.nn.conv2d(
    input = x,
    filters = y,
    strides =(1, 1, 1, 1),
    padding = "VALID"
)

# print(out)
out = tf.nn.bias_add(out, tf.constant([2, 3], dtype=float))
print(out)


tf.Tensor(
[[[37. 12.]
  [47. 16.]]], shape=(1, 2, 2), dtype=float32)
tf.Tensor(
[[[[1. 1.]
   [3. 1.]]

  [[2. 1.]
   [4. 1.]]]], shape=(1, 2, 2, 2), dtype=float32)
tf.Tensor(
[[[[39. 15.]
   [49. 19.]]]], shape=(1, 1, 2, 2), dtype=float32)


In [ ]:
import numpy as np
x = np.array([[[3, 6, 17, 15],[13, 15, 6, 12],[9, 1, 2, 7]],[[10, 19, 3, 6],[0, 6, 12, 16],[11, 8, 7, 9]]])
x = np.transpose(x, (1, 2, 0))
# print(x)

y = np.array([[1, 2, 3], [4, 5, 6]])
y = np.reshape(y, (1, 1, 2, 3))
y = np.transpose(y, (0, 1, 3, 2))
print(y)

[[[[1 4]
   [2 5]
   [3 6]]]]


In [ ]:
def sinc(t):
  return tf.where(t == 0, tf.constant(1., dtype=t.dtype), tf.math.sin(t) / t)


def kernel_for_upsample_and_downsample_2(zeros=56):
	win = tf.signal.hann_window(window_length = 4*zeros+1, periodic=False)
	# print(win)
	winodd = win[1::2]
	t = tf.linspace(-zeros+0.5, zeros-0.5, 2*zeros)
	t *= math.pi
	# print(t.dtype)
	kernel = tf.reshape(sinc(t)*winodd, [-1, 1, 1])
	return kernel

def upsample(x, zeros=56):
	batch, sample, ch = x.shape
	kernel = kernel_for_upsample_and_downsample_2(zeros)
	# print(kernel)
	out = tf.reshape(x, [-1, sample, 1])
	out = tf.nn.conv1d(input=out, filters=kernel, stride=1, padding="SAME")
	print(out)
	# out = out[:,1:,:]
	out = tf.reshape(out, [-1, sample, ch])
	y = tf.stack([x, out], axis=-2)
	# print(tf.reshape(y, [-1, sample * 2, 1]))
	return tf.reshape(y, [-1, sample * 2, 1])

In [ ]:
x = np.array([0.23, 0.55, 0.65, 0.4, 0.33, 0.4])
# x = np.random.rand(1, 160000, 1)
x = tf.constant(x, dtype=tf.float32)
x = tf.reshape(x, [1, 6, 1])
np.set_printoptions(precision = 15)
print(upsample(x))

tf.Tensor(
[[[0.40757713]
  [0.63612163]
  [0.5617281 ]
  [0.29118615]
  [0.4287346 ]
  [0.20211749]]], shape=(1, 6, 1), dtype=float32)
tf.Tensor(
[[[0.23      ]
  [0.40757713]
  [0.55      ]
  [0.63612163]
  [0.65      ]
  [0.5617281 ]
  [0.4       ]
  [0.29118615]
  [0.33      ]
  [0.4287346 ]
  [0.4       ]
  [0.20211749]]], shape=(1, 12, 1), dtype=float32)


In [ ]:
# print(np.pi)
# print(math.pi)
# np.set_printoptions(precision = 1)
# print(np.cos(2.34))
x = 2.34
y = math.cos(x)
# print(type(x))
# print((y))
from tensorflow.python.framework import constant_op
from tensorflow.python.framework import dtypes
from tensorflow.python.ops import math_ops
cos_arg = constant_op.constant(2 * np.pi, dtype=dtypes.float32) * 1 / 224
print(math_ops.cos(cos_arg).numpy())
print(math_ops.cast(0.5 - 0.5 * math_ops.cos(cos_arg), dtype=dtypes.float32))
print(math.cos(2.0 * np.pi * 1 / 224))
print((0.5 - 0.5 * math.cos(2.0 * np.pi * 1 / 224)))

0.9996066
tf.Tensor(0.00019669533, shape=(), dtype=float32)
0.9996066263830529
0.00019668680847356734


In [ ]:
# han = tf.signal.hann_window(window_length = 4*56+1, periodic=False)
# print(han)

In [ ]:
def _compute_carry_and_output_fused(z, c_tm1):
        z0, z1, z2, z3 = z
        i = K.sigmoid(z0)
        f = K.sigmoid(z1)
        c = f * c_tm1 + i * K.tanh(z2)
        o = K.sigmoid(z3)
        return c, o

In [ ]:
import tensorflow.keras.backend as K
import numpy as np
import tensorflow as tf
inputs = tf.constant(np.array([[1, 2, 3, 4]], dtype=np.float32))
h_tm1 = tf.constant(np.zeros((1, 4), dtype=np.float32))
c_tm1 = tf.constant(np.zeros((1, 4), dtype=np.float32))
kernel = tf.constant(np.ones((4, 16), dtype=np.float32))
recurrent_kernel = tf.constant(np.ones((4, 16), dtype=np.float32))
bias = tf.constant(np.zeros((16), dtype=np.float32))
# recurrent_kernel = np.array([[2, 2, 2, 2]])
z = K.dot(inputs, kernel)
z += K.dot(h_tm1, recurrent_kernel)
z = K.bias_add(z, bias)
z = tf.split(z, num_or_size_splits=4, axis=1)
c, o = _compute_carry_and_output_fused(z, c_tm1)
h = o * K.tanh(c)
print(h)
print(c)
inputs = h
h_tm1 = h
c_tm1 = c
# print(inputs)
# print(h_tml)
z = K.dot(inputs, kernel)
z += K.dot(h_tm1, recurrent_kernel)
z = K.bias_add(z, bias)
z = tf.split(z, num_or_size_splits=4, axis=1)
c, o = _compute_carry_and_output_fused(z, c_tm1)
h = o * K.tanh(c)
print(h)
print(c)


tf.Tensor([[0.76154053 0.76154053 0.76154053 0.76154053]], shape=(1, 4), dtype=float32)
tf.Tensor([[0.9999546 0.9999546 0.9999546 0.9999546]], shape=(1, 4), dtype=float32)
tf.Tensor([[0.9615303 0.9615303 0.9615303 0.9615303]], shape=(1, 4), dtype=float32)
tf.Tensor([[1.9954343 1.9954343 1.9954343 1.9954343]], shape=(1, 4), dtype=float32)


In [ ]:
import tensorflow as tf
import numpy as np
inputs = tf.keras.layers.Input(shape=(2, 2))
outputs = tf.keras.layers.RNN(tf.keras.layers.LSTMCell(units=2, kernel_initializer=tf.keras.initializers.Constant(value=1.0)), return_sequences = True)(inputs)
model = tf.keras.models.Model(inputs=inputs, outputs=outputs)

k1 = np.array([[1, 2, 3, 4, 5, 6, 7, 8], [5, 6, 7, 8, 9, 10, 11, 12]])
k2 = np.array([[1, 2, 3, 4, 5, 6, 7, 8], [5, 6, 7, 8, 9, 10, 11, 12]])
# lstm = tf.keras.Sequential()
# lstm.add()
inp = tf.constant(np.array([[1, 2], [5, 100]], dtype=np.float32))
inp = tf.reshape(inp, shape=(1, 2, 2))
# lstm.compile(optimizer='sgd', loss='mse')
print(model.summary())
for layer in model.layers:
  for par in layer.trainable_weights:
    if "kernel:0" in par.name:
      par.assign(k1)
    elif "recurrent_kernel:0" in par.name:
      par.assign(k2)
    else:
      par.assign(np.ones(par.shape))
    # print(par)
for layer in model.layers:
  for par in layer.trainable_weights:
    print(par)
x = model(inp)
print(x)

Model: "model_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_26 (InputLayer)       [(None, 2, 2)]            0         
                                                                 
 rnn_32 (RNN)                (None, 2, 2)              40        
                                                                 
Total params: 40
Trainable params: 40
Non-trainable params: 0
_________________________________________________________________
None
<tf.Variable 'rnn_32/lstm_cell_32/kernel:0' shape=(2, 8) dtype=float32, numpy=
array([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.],
       [ 5.,  6.,  7.,  8.,  9., 10., 11., 12.]], dtype=float32)>
<tf.Variable 'rnn_32/lstm_cell_32/recurrent_kernel:0' shape=(2, 8) dtype=float32, numpy=
array([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.],
       [ 5.,  6.,  7.,  8.,  9., 10., 11., 12.]], dtype=float32)>
<tf.Variable 'rnn_32/lstm_cell_32/bias:0' shape=(8,) dt

In [14]:
import numpy as np
import tensorflow as tf
input = tf.constant(np.ones((1, 624, 1024)), dtype=float)
filter = tf.constant(np.ones((8, 512, 1024)), dtype=float)
# cc = tf.keras.layers.Conv1DTranspose(
# 					filters=512, kernel_size=8, strides=4, kernel_initializer=tf.keras.initializers.Constant(1), bias_initializer=tf.keras.initializers.Constant(1)
# 				)
# out = cc(input)
out = tf.nn.conv1d_transpose(input, filter, strides=4, output_shape=[1, 2500, 512], padding="VALID")
# out = tf.nn.conv1d(input=input, filters=filter, stride=1, padding="SAME")
print(out[0][1222])

tf.Tensor(
[2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048.
 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048.
 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048.
 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048.
 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048.
 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048.
 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048.
 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048.
 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048.
 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048.
 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048.
 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048.
 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048. 2048.
 2048. 2048. 2048. 2048. 2048. 2048. 204